# Setup

In [ ]:
# Set Up Environment
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
from kaggle_environments import evaluate, make
env = make("halite", configuration={ "episodeSteps": 400 }, debug=True)
print (env.configuration)
conf = Configuration(env.configuration)

# Submission

In [ ]:
# automatically reimport when file has changed
%load_ext autoreload
%autoreload 2
from submission import agent, internals

# Run a game

In [ ]:
env.run(["submission.py", "random","random","random"])
env.render(mode="ipython", width=800, height=600)

# DEBUG

In [ ]:
trainer = env.train([None, "random"])
observation = trainer.reset()

In [ ]:
observation, reward, done, info = trainer.step(my_action)
env.render(mode="ipython", header=False, controls=False)

In [ ]:
my_action = agent(observation, env.configuration)
print("action :", my_action)
board = Board(observation, env.configuration)
obs = Observation(observation)

In [ ]:
pd.DataFrame(internals.halite_map).style.background_gradient(cmap='Blues')

In [ ]:
threat_map = internals.threat_map.transpose()
threat_map = gaussian_filter(threat_map, sigma=2)
sns.heatmap(threat_map, cmap='RdYlGn', linewidths=0.5, annot=False)

In [ ]:
halite_map = internals.halite_map
halite_map = gaussian_filter(halite_map, sigma=1)
sns.heatmap(halite_map, cmap='Blues', linewidths=0.5, annot=False)

In [ ]:
final_map = halite_map*3 + threat_map
sns.heatmap(final_map, cmap='RdYlGn', linewidths=0.5, annot=False)

In [ ]:
env.render(mode="ipython", header=False, controls=False)